## Overview of Assignment 4

This assignment focuses on exploring and implementing advanced concepts and techniques in information retrieval. The primary objectives are to build Retrieval Augumentation Generation, and learn about Language Models

## Enter your details below

## Name

In [ ]:
Riham Otman

## Banner ID

In [ ]:
B00887629

## GitHub Link of your Assingment 4

https://github.com/Riham-Otman/csci4141-assignment4-rag

## Q1 : Setting up the libraries and the environment

In [1]:
pip install langchain faiss-cpu openai tqdm pandas jupyterlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling jupyter-client-6.1.12:
      Successfully uninstalled jupyter-client-6.1.12
  Attempting uninstall: jupyter-server
    Found existing installation: jupyter-server 1.16.0
    Uninstalling jupyter-server-1.16.0:
      Successfully uninstalled jupyter-server-1.

In [2]:
pip freeze > requirements.txt


## Q2:  Data Preprocessing and Model Selection

## Q3: Implementing RAG using LangChain for different queries

## Q4 : Modify and evaluate the different components of RAG

## Q5: Selecting and implementing a pretrained model for a new task